In [433]:
import pandas as pd
import re

In [434]:
data = pd.read_csv("/work/hack/train_dataset_train.csv", sep=";")

In [435]:
rename_columns = {
    "Текст инцидента": "text",
    "Тема": "subject",
    "Исполнитель": "executor",
    "Группа тем": "group_subject"
}
data = data.rename(columns=rename_columns)

In [436]:
data.describe()

,executor,group_subject,text,subject
count,23128,23128,23128,23128
unique,10,26,22656,195
top,Министерство здравоохранения,Здравоохранение/Медицина,'Сообщение без текста,Дети и многодетные семьи
freq,5765,4673,134,2008


In [437]:
data['text'].value_counts()[:3].index

Index([''Сообщение без текста', ''Здравствуйте', '''], dtype='object')

In [438]:
dataset = data[['executor', 'text']]
dataset = dataset.rename(columns={"executor": "label"})

In [439]:
print(len(data))
data['text'] = data['text'].apply(lambda x: re.sub(r'http\S+', '', x)) 
data['text'] = data['text'].apply(lambda x: re.sub('<[^<]+?>', '', x))
data = data[data['text'] != "'Сообщение без текста"]
data['text'] = data['text'].apply(lambda x: x.replace("'", ""))
replics_to_remove = [
    "Да", "Начать", "???", "Писала", "Да.", "да", "Спасибо за ответ",
    "Здравствуйте", "Да, подтверждаю", "Добрый день",
    "Да подтверждаю", "Добрый день!", "Да, подтверждаю.",
    "Здравствуйте!", "Добрый день.", "Доброе утро!",
    "Да,подтверждаю", "Принять и продолжить 👌",
    "Ау", "Это что?", "Ива 1", "Везде",
    "Жду ответ", "Пропустить", "Да, потверждаю",
    "Спрашиваю )", "Здраствуйте!", "Здраствуйте,",
    "добрый день", "Очень грустно", "Добавляю фото",
    "август 2022", "Дополнение...", "Не работает",
    "Добрый вечер!", "Добрый день!!", "Здравствуйте.",
    "Я подтверждаю", "Здравствуйте,", "Да подтвержаю",
    "Жду ответа", "> [Анонимно.]", "Квест..",
    "Это что такое", "Детям тоже?", "Даже в 14 лет?",
    "VID_20220912_095100.mp4", "05.09.2022",
    "Да подтверждаю!", "Это вообще норм", "Вечная проблема",
    "Чайка тоже нет😐", "Вы мне ответите?", "Пять минут назад",
    "@samaya547, Наталья", "Да , подтверждаю .",
    "Интересно это правда или как?", "Посылаю вам фотографии"
]
data = data[~data['text'].isin(replics_to_remove)]
data = data[~data['text'].str.isspace()]
data = data[data['text'] != ""]
data = data.drop_duplicates()
v = data['text'].value_counts()
data = data[~data['text'].isin(v.index[v.gt(1)])]
print(len(data))

23128
22543


In [440]:
from sklearn.model_selection import train_test_split

data['for_strat'] = data['group_subject'] + data['subject']
train, test = train_test_split(data, stratify=data['for_strat'], test_size=0.2)
train = train.drop(columns=['for_strat'])
test = test.drop(columns=['for_strat'])

In [441]:
train.to_csv("train_dataset.csv", index=False)
test.to_csv("test_dataset.csv", index=False)

In [ ]:
train